In [9]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import os
import sys
from os.path import join 
repo_dir = '/content/drive/MyDrive/gong-metaphor-detection'

In [ ]:
# pip install requirements (takes a while)
#!cd drive/MyDrive/Repos/gong-metaphor-detection/; pip install -r requirements.txt

In [ ]:
!pip install 'transformers==3.5.1'

In [ ]:
!pip install torch==1.4.0

In [ ]:
!pip install scikit-learn

In [11]:
!cd drive/MyDrive/Repos/gong-metaphor-detection/; python3 run_metaphor_detection.py \
--data_dir data/VUA \
--model_type roberta \
--model_name_or_path roberta-large \
--output_dir output/VUA/model/ \
--dataset VUA \
--max_seq_length 256 \
--do_predict \
--do_lower_case \
--use_pos \
--overwrite_cache

03/15/2022 05:53:36 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
03/15/2022 05:53:42 - INFO - modeling_roberta_metaphor -   hidden_size: 1024, pos_dim: 8, feature_dim: 128
03/15/2022 05:53:42 - INFO - modeling_roberta_metaphor -   classifier dim: 1032
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForMetaphorDetection: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMetaphorDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMetaphorDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertF

# Eval results on test data

03/15/2022 05:56:05 - INFO - __main__ -   ***** Eval results *****

03/15/2022 05:56:05 - INFO - __main__ -     accuracy = 0.948101644245142

03/15/2022 05:56:05 - INFO - __main__ -     f1 = 0.7892521851731952

03/15/2022 05:56:05 - INFO - __main__ -     loss = 0.0

03/15/2022 05:56:05 - INFO - __main__ -     precision = 0.7976443644691641

03/15/2022 05:56:05 - INFO - __main__ -     recall = 0.7810347589300016

03/15/2022 05:56:05 - INFO - __main__ -   Saving predictions to output/VUA/model/test_labels.txt

In [ ]:
import os
import ast
from sklearn.metrics import precision_score, recall_score, f1_score
# need to be in repo root before importing from core
from core.data.gao_data import ExperimentData
from core.data.io_util import write_dict_to_json

### INPUT
input_data_path = os.path.join("resources", "metaphor-in-context", "data")
predictions_data_path = os.path.join("output", "VUA", "modeltest")

In [ ]:
gao_data = ExperimentData(input_data_path)
gao_data.read_vua_seq_data()

In [ ]:
len(gao_data.vua_seq_formatted_test)

In [ ]:
print(len(gao_data.vua_seq_formatted_test))
gao_data.vua_seq_formatted_test[10]

2694


['a3m-fragment02',
 '58',
 'Trams disappeared from most British cities in the 1950s .',
 '[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]',
 "['NOUN', 'VERB', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT']",
 'Trams disappeared from most British cities M_in the 1950s .',
 'news']

In [ ]:
test_pred_labels = []
f = open(predictions_data_path + "/test_labels.txt", "r")
for labels in f:
  labels_lst_str = labels.split()
  labels_lst = [int(lbl) for lbl in labels_lst_str] 
  test_pred_labels.append(labels_lst)

In [ ]:
test_pred_labels[10]

[0, 1, 1, 0, 0, 0, 1, 0, 0, 0]

In [ ]:
test_true_labels = [ast.literal_eval(sentence[3]) for sentence in gao_data.vua_seq_formatted_test]
test_true_labels[10]

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [ ]:
for ind in range(2694):
  assert len(test_true_labels[ind]) == len(test_pred_labels[ind]), "Unequal lengths"